In [1]:
import numpy as np
from functools import wraps

In [2]:
class SimpleTensor:
    def __init__(self, data, requires_grad=False, parents=None, creation_op=None):
        self.data = np.array(data) # hold the data
        self.requires_grad = requires_grad # flag to check if gradient is required
        self.parents = parents or [] # hold the parents
        self.grad = None # hold the gradient (this should only stay None if requires_grad is False)
        self.creation_op = creation_op # hold the operation that created the tensor

        if self.requires_grad: # if gradient is required, zero initialize the gradient
            self.grad = np.zeros_like(self.data)

    @property
    def backward_ops(self):
        """
        I did this to clearly see what's implemented and what's not
        """
        ops = {
            "add": self.backward_add,
            "mul": self.backward_mul
        }
        return ops
    
    def make_tensor(func):
        """
        Decorator to convert the 'other' arg to a tensor if its not already a tensor
        """
        @wraps(func) # does this line matte?
        def wrapper(self, other):
            if not isinstance(other, SimpleTensor):
                other = SimpleTensor(other)
            return func(self, other)
        return wrapper

    @make_tensor
    def __add__(self, other):
        data = np.add(self.data, other.data)
        return SimpleTensor(data, requires_grad=(self.requires_grad or other.requires_grad), parents=[self, other], creation_op="add")
    
    def backward_add(self):
        self.parents[0].backward(self.grad)
        self.parents[1].backward(self.grad)
    
    @make_tensor
    def __mul__(self, other):
        data = np.multiply(self.data, other.data)
        return SimpleTensor(data, requires_grad=(self.requires_grad or other.requires_grad), parents=[self, other], creation_op="mul")
    
    def backward_mul(self):
        grad_wrt_first_parent = self.grad * self.parents[1].data
        grad_wrt_second_parent = self.grad * self.parents[0].data
        self.parents[0].backward(grad_wrt_first_parent)
        self.parents[1].backward(grad_wrt_second_parent)

    def backward(self, grad=None):
        if not self.requires_grad: # if gradient is not required, return
            return
        
        if grad is None:  # if we call backward without passing a gradient, initialize the gradient to 1
            grad = np.ones_like(self.data)

        if self.grad is None:  # initialize self.grad if it's None
            self.grad = grad
        else:
            self.grad += grad  # accumulate gradient

        if self.creation_op is None: # if the tensor was created by the user
            return
        
        # run the appropriate backward operation
        backward_op = self.backward_ops.get(self.creation_op, None)
        if backward_op is not None:
            backward_op() # call it
        else:
            raise NotImplementedError("Only addition and multiplication implemented")
        
    def __repr__(self):
        return f"SimpleTensor(data={self.data}, requires_grad={self.requires_grad}, grad={self.grad})"

In [3]:
m1 = SimpleTensor([[1, 2], [3, 4]], requires_grad=True)
m2 = SimpleTensor([[5, 6], [7, 8]], requires_grad=True)
v = SimpleTensor([9, 10], requires_grad=True)

# multiplying two tensors
m3 = m1 * m2 * 2

m3.backward()

print(m1.grad)
print(m2.grad)

[[10 12]
 [14 16]]
[[2 4]
 [6 8]]
